In [ ]:
!pip install -qU educhain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## A custom model that matches the schema of current Supabase Table

In [ ]:
  # Custom Model
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field, validator
from datetime import datetime
import uuid
class Optioncustom(BaseModel):
    text: str = Field(description="The text of the option.")
    correct: str = Field(description="Whether the option is correct or not. Either 'true' or 'false'")


class MCQcustom(BaseModel):
    question: str = Field(description="The quiz question")
    options: List[Optioncustom] = Field(description="The possible answers to the question. The list should contain 4 options.")
    explanation: str = Field(default=None, description="Explanation of the question")
    blooms_level: str = Field(default=None, description="The Bloom's taxonomy level of the question")
    difficulty_level: str = Field(default=None, description="The difficulty level of the question. Can be 'easy', 'medium' or 'hard' ")
    difficulty_rating: int = Field(ge=1, le=3, description="The difficulty rating of the question (1-5)")
    metadata: Dict[str, Any] = Field(default={}, description="Additional metadata for the question.")
    topic: str = Field(default=None, description="The topic of the question")
    subject: str = Field(default=None, description="The subject of the question")
    created_at: datetime = Field(default_factory=datetime.now, description="Timestamp when the question was created")
    modified_at: datetime = Field(default_factory=datetime.now, description="Timestamp when the question was last modified")
    uuid: str = Field(default_factory=lambda: str(uuid.uuid4()), description="Unique identifier for the question")
    grade: int = Field(default= 5, description="Grade level for the question (default: 7)")

    def json(self, **kwargs):
        """
        Override json method to serialize datetime objects properly.
        """
        # Convert datetime objects to ISO format strings
        self.created_at = self.created_at.isoformat()
        self.modified_at = self.modified_at.isoformat()
        return super().json(**kwargs)

    @property
    def correct_answer(self):
        for option in self.options:
            if option.correct.lower() == 'true':
                return option.text
        return None

    def show(self):
        options_str = "\n".join(f"  {chr(65 + i)}. {option.text}" for i, option in enumerate(self.options))
        print(f"Question: {self.question}\nOptions:\n{options_str}")
        print(f"Correct Answer: {self.correct_answer}")
        print(f"Explanation: {self.explanation}")
        print(f"Bloom's Level: {self.blooms_level}")
        print(f"Difficulty Level: {self.difficulty_level}")
        print(f"Difficulty Rating: {self.difficulty_rating}")
        print(f"Metadata: {self.metadata}\n")


class MCQListcustom(BaseModel):
    questions: List[MCQcustom]

    def show(self):
        print("MCQs:\n")
        for i, mcq in enumerate(self.questions, start=1):
            print(f"Question {i}:")
            mcq.show()

## A custom template that can be used for question generation

In [ ]:
from educhain import qna_engine

custom_template =  """
  You are an Academic AI assistant tasked with generating multiple-choice questions on various topics.
  Don't restrict the questions to start with 'which','what' and 'how'.Frame the questions such that the user doesnt find the questions to be repetative even if they are different.
  Include a few fill in the blank questions as well.
  Give me {num} number of questions based on the following parameters:
  Subject: {subject}
  Topic: {topic}
  Subtopic: {subtopic}
  Learning Objective: {learning_objective}

  NOTE: The questions should be fun and easy to answer.

  Please keep the following points in mind before creating questions
  1. Each question should have 4 options, and only 1 correct out of them.
  2. Bloom's level can be "Knowing", "Understanding", "Analyzing", "Applying", "Evaluating", "Creating"
  3. Make sure to add all kinds of difficulty level: easy, medium and hard.
  4. Metadata should contain 'subject', 'topic', 'subtopic' and 'learning_objective'
  5. Generate a concise explanation for each question.

  Please ensure variety in questions.
  The response should be in JSON format. \n {format_instructions}
  """


result = qna_engine.generate_mcq(
            num = 1,
             subject = "Maths",
             topic = "Addition of Numbers",
             subtopic = "Additions",
             learning_objective = "To help learn fundamentals of addition",
    prompt_template = custom_template,
    response_model = MCQListcustom
)

result

MCQListcustom(questions=[MCQcustom(question='What is the sum of 5 + 3?', options=[Optioncustom(text='7', correct='true'), Optioncustom(text='9', correct='false'), Optioncustom(text='6', correct='false'), Optioncustom(text='8', correct='false')], explanation='Adding 5 and 3 gives the result of 8.', blooms_level='Understanding', difficulty_level='easy', difficulty_rating=1, metadata={'subject': 'Maths', 'topic': 'Addition of Numbers', 'subtopic': 'Additions', 'learning_objective': 'To help learn fundamentals of addition'}, topic=None, subject=None, created_at=datetime.datetime(2022, 10, 20, 8, 0, tzinfo=TzInfo(UTC)), modified_at=datetime.datetime(2022, 10, 20, 8, 0, tzinfo=TzInfo(UTC)), uuid='1234', grade=5)])

In [ ]:
result.show()

MCQs:

Question 1:
Question: What is the sum of 5 + 3?
Options:
  A. 7
  B. 9
  C. 6
  D. 8
Correct Answer: 7
Explanation: Adding 5 and 3 gives the result of 8.
Bloom's Level: Understanding
Difficulty Level: easy
Difficulty Rating: 1
Metadata: {'subject': 'Maths', 'topic': 'Addition of Numbers', 'subtopic': 'Additions', 'learning_objective': 'To help learn fundamentals of addition'}



## Function to delete generated Questions

In [ ]:
def display_mcqs(mcq_list_instance):
    """
    Displays a list of MCQs in a formatted string and allows deletion of questions.

    :param mcq_list_instance: An instance of MCQList containing a list of MCQ objects.
    :return: A formatted string listing all the questions.
    """
    formatted_mcqs = ""
    for index, mcq in enumerate(mcq_list_instance.questions):
        formatted_mcqs += f"{index + 1}. {mcq.question}\n- Options:\n{', '.join(option.text for option in mcq.options)}\n- Explanation: {mcq.explanation}\n- Bloom's Level: {mcq.blooms_level}\n- Difficulty Level: {mcq.difficulty_level}\n- Metadata: {mcq.metadata}\n\n"

    print(formatted_mcqs)
    delete_question = input("Press 1 to delete a question. Enter the question number to delete it: ")
    if delete_question.isdigit():
        question_number = int(delete_question) - 1  # Adjust for zero-based indexing
        if 0 <= question_number < len(mcq_list_instance.questions):
            del mcq_list_instance.questions[question_number]
            print(f"Question {question_number + 1} has been deleted.")
        else:
            print("Invalid question number.")
    else:
        print("Please enter a valid question number.")


In [ ]:
# Assuming output_questions is an instance of MCQList
formatted_mcqs = display_mcqs(result)
print(formatted_mcqs)


1. What is the sum of 5 + 3?
- Options:
7, 9, 6, 8
- Explanation: Adding 5 and 3 gives the result of 8.
- Bloom's Level: Understanding
- Difficulty Level: easy
- Metadata: {'subject': 'Maths', 'topic': 'Addition of Numbers', 'subtopic': 'Additions', 'learning_objective': 'To help learn fundamentals of addition'}


Press 1 to delete a question. Enter the question number to delete it: 0
Invalid question number.
None


## Push to DB

In [ ]:
!pip install -qU supabase

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.4 MB/s eta 0:00:00


## Connect Databse

In [ ]:


%env SUPABASE_URL = "YOUR URL"

%env SUPABASE_KEY = "YOUR KEY"

env: SUPABASE_URL="YOUR URL"
env: SUPABASE_KEY="YOUR KEY"


In [ ]:
import os
import supabase

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_KEY")

# Create Supabase client
supabase_client = supabase.create_client(supabase_url, supabase_key)

## Shuffle the Options

In [ ]:
import random

# Define a function to shuffle options within each question
def shuffle_options(mcq_list):
    shuffled_questions = []
    for mcq in mcq_list.questions:
        # Shuffle the options for each question}
        random.shuffle(mcq.options)
        shuffled_questions.append(mcq)
    return shuffled_questions

## Push to DB

In [ ]:
from datetime import datetime
import uuid

def push_to_db(output_questions: MCQListcustom):
    # Assuming shuffle_options is a function that shuffles the options of each question
    shuffled_mcq_list = shuffle_options(result)
    for question in shuffled_mcq_list:
        # Extract topic and subject from metadata
        metadata = question.metadata
        topic = metadata.get('topic')
        subject = metadata.get('subject')

        # Populate topic and subject fields
        question.topic = topic
        question.subject = subject

        # Populate date_created and date_modified fields
        question.created_at = datetime.now()  # Set the creation date to the current timestamp
        question.modified_at = datetime.now()  # Set the modification date to the current timestamp

        try:
            # Serialize MCQ object to a dictionary for insertion
            question_dict = question.dict()

            # Generate a UUID for the question
            question_uuid = str(uuid.uuid4())
            question_dict['uuid'] = question_uuid

            # Convert datetime objects to ISO format strings for serialization
            question_dict['created_at'] = question.created_at.isoformat()
            question_dict['modified_at'] = question.modified_at.isoformat()

            # Insert the question into the database
            supabase_client.table("test_educhain").insert(question_dict).execute()
        except Exception as e:
            print(f"An error occurred: {e}")


In [ ]:
push_to_db(result)

In [ ]:
result

MCQListcustom(questions=[MCQcustom(question='What is the sum of 5 + 3?', options=[Optioncustom(text='9', correct='false'), Optioncustom(text='8', correct='false'), Optioncustom(text='7', correct='true'), Optioncustom(text='6', correct='false')], explanation='Adding 5 and 3 gives the result of 8.', blooms_level='Understanding', difficulty_level='easy', difficulty_rating=1, metadata={'subject': 'Maths', 'topic': 'Addition of Numbers', 'subtopic': 'Additions', 'learning_objective': 'To help learn fundamentals of addition'}, topic='Addition of Numbers', subject='Maths', created_at=datetime.datetime(2024, 5, 24, 17, 6, 42, 581179), modified_at=datetime.datetime(2024, 5, 24, 17, 6, 42, 581225), uuid='1234', grade=5)])